# Examples of running IHPF 

In [2]:
import IHPF
import scanpy as sc

/home/mw4315/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
from scipy.sparse import coo_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

In [1]:
datasets = ['humanpancreas','10Xmouse','10Xpbmc','mixedpbmc']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    adataobs = adata.obs.copy()
    no_cell_types = len(adataobs['actual'].unique())
    adataobs.reset_index(inplace=True)
    Xlist = list()
    for i,df in adataobs.groupby('batch'):
        batchidx = df.index
        Xlist.append(coo_matrix(adata.X[batchidx,:]))
    model = IHPF.scIHPF(no_cell_types,max_iter=500)
    model.fit(Xlist)
    adata.obsm['IHPF_{}'.format(no_cell_types)] = np.concatenate(model.cell_scores(),axis=0)
    adata.varm['IHPF_{}'.format(no_cell_types)] = model.shared_gene_scores()
    kmeans_cell = KMeans(n_clusters=no_cell_types, random_state=0).fit(normalize(adata.obsm['IHPF_{}'.format(no_cell_types)]))
    adata.obs['IHPF_{}_kmeans_normalised'.format(no_cell_types)] = kmeans_cell.labels_
    print(adata)
    

NameError: name 'sc' is not defined

In [31]:
adata.obs

,actual,batch,IHPF_kmeans_normalised,HPF_kmeans_normalised,INMF_kmeans_normalised,IPCA_kmeans_normalised,IHPF_2_kmeans_normalised
0,293t,0,1,0,1,0,0
1,293t,0,1,0,1,0,0
2,293t,0,1,0,1,0,0
3,293t,0,1,0,1,0,0
4,293t,0,1,0,1,0,0
...,...,...,...,...,...,...,...
9525,jurkat,2,0,1,0,0,1
9526,293t,2,1,1,1,0,0
9527,293t,2,1,1,1,0,0
9528,293t,2,1,1,1,0,0


In [32]:
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score
datasets = ['humanpancreas','10Xmouse','newpbmc','hpfpbmc']
batchAMI = dict()
actualAMI = dict()

In [33]:
for dataset in datasets:
    batch = dict()
    actual = dict()
    # adata = sc.read('../Real/scanorama/{}.h5ad'.format(dataset))
    for method in ['IHPF_2','IHPF','HPF','INMF','IPCA']:
        batch[method] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['{}_kmeans_normalised'.format(method)])
        actual[method] = adjusted_mutual_info_score(adata.obs['actual'],adata.obs['{}_kmeans_normalised'.format(method)])
    batch['actual'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    actual['batch'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    batchAMI[dataset] = batch
    actualAMI[dataset] = actual

In [34]:
pd.DataFrame(batchAMI)

,humanpancreas,10Xmouse,newpbmc,hpfpbmc
IHPF_2,0.498353,0.498353,0.498353,0.498353
IHPF,0.498353,0.498353,0.498353,0.498353
HPF,0.744954,0.744954,0.744954,0.744954
INMF,0.288737,0.288737,0.288737,0.288737
IPCA,0.738750,0.738750,0.738750,0.738750
actual,0.498373,0.498373,0.498373,0.498373


In [35]:
pd.DataFrame(actualAMI)

,humanpancreas,10Xmouse,newpbmc,hpfpbmc
IHPF_2,0.976076,0.976076,0.976076,0.976076
IHPF,0.976076,0.976076,0.976076,0.976076
HPF,-0.000067,-0.000067,-0.000067,-0.000067
INMF,0.226908,0.226908,0.226908,0.226908
IPCA,0.447701,0.447701,0.447701,0.447701
batch,0.498373,0.498373,0.498373,0.498373
